# Section 2: Data Mining
## Task 3: Classification (Part A, 10 Marks) & Association Rule Mining (Part B, 10 Marks)
This notebook covers: (A) Decision Tree + KNN classification with metrics & visualization; (B) synthetic transactional data generation and Apriori association rule mining with analysis.

In [ ]:
# 1. Imports & Data Reload (re-using Task 1 preprocessing steps)
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import json, random

sns.set_theme(style='whitegrid', context='notebook')
ARTIFACT_DIR = Path('artifacts')
ARTIFACT_DIR.mkdir(exist_ok=True)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
DATA_OPTION = 'iris'  # could toggle synthetic pattern if expanded
print('Data option:', DATA_OPTION)

def load_iris_dataframe():
    iris = load_iris(as_frame=True)
    df = iris.frame.copy()
    df.rename(columns={'target':'class'}, inplace=True)
    mapping = {i:name for i,name in enumerate(iris.target_names)}
    df['class'] = df['class'].map(mapping)
    return df

df = load_iris_dataframe()
feature_cols = [c for c in df.columns if c != 'class']
X = df[feature_cols].copy()
y = df['class'].copy()
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_cols)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=RANDOM_SEED)
print('Train/Test shapes:', X_train.shape, X_test.shape)

In [ ]:
# 2. Decision Tree Classifier (Primary)
dt = DecisionTreeClassifier(random_state=RANDOM_SEED, max_depth=4)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
acc_dt = accuracy_score(y_test, y_pred_dt)
prec_dt, rec_dt, f1_dt, _ = precision_recall_fscore_support(y_test, y_pred_dt, average='macro', zero_division=0)
print(f'Decision Tree -> Acc: {acc_dt:.3f} | Precision: {prec_dt:.3f} | Recall: {rec_dt:.3f} | F1: {f1_dt:.3f}')
print('Classification report (Decision Tree):')
print(classification_report(y_test, y_pred_dt, zero_division=0))
plt.figure(figsize=(10,6))
plot_tree(dt, feature_names=feature_cols, class_names=sorted(y.unique()), filled=True, rounded=True)
plt.title('Decision Tree (max_depth=4)')
plt.tight_layout()
plt.savefig(ARTIFACT_DIR / 'decision_tree_plot.png', dpi=150)
plt.show()
print('Saved tree plot to artifacts/decision_tree_plot.png')

In [ ]:
# 3. KNN Classifier (k=5) for Comparison
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
acc_knn = accuracy_score(y_test, y_pred_knn)
prec_knn, rec_knn, f1_knn, _ = precision_recall_fscore_support(y_test, y_pred_knn, average='macro', zero_division=0)
print(f'KNN (k=5) -> Acc: {acc_knn:.3f} | Precision: {prec_knn:.3f} | Recall: {rec_knn:.3f} | F1: {f1_knn:.3f}')
print('Classification report (KNN):')
print(classification_report(y_test, y_pred_knn, zero_division=0))
comparison_df = pd.DataFrame([
    {'model':'DecisionTree','accuracy':acc_dt,'precision':prec_dt,'recall':rec_dt,'f1':f1_dt},
    {'model':'KNN(k=5)','accuracy':acc_knn,'precision':prec_knn,'recall':rec_knn,'f1':f1_knn}
]).set_index('model')
display(comparison_df)

### 4. Model Comparison & Rationale
The table above summarizes macro-averaged metrics. Generally, KNN often performs strongly on Iris due to well-separated species in petal space, while a constrained-depth tree provides interpretability. If their performance is similar, preference may lean toward the tree for explainability (clear decision paths). If KNN edges out in F1/accuracy, it suggests local neighborhood structure captures subtle class boundaries better than hierarchical splits. In production, pruning, cross-validation, or ensembling (e.g., Random Forest) could further improve robustness.

In [ ]:
# 5. Part B: Synthetic Transactional Data Generation
item_pool = ['milk','bread','butter','cheese','eggs','beer','diapers','apples','bananas','cereal',
              'chicken','rice','pasta','tomatoes','onions','yogurt','chips','soda','coffee','tea']
n_transactions = 40
rng = np.random.default_rng(RANDOM_SEED)
transactions = []
for _ in range(n_transactions):
    basket_size = rng.integers(3,9)
    basket = rng.choice(item_pool, size=basket_size, replace=False).tolist()
    # Inject a few patterns: milk-bread-butter & beer-diapers, coffee-tea more frequent
    if rng.random() < 0.4:
        for it in ['milk','bread']:
            if it not in basket: basket.append(it)
    if rng.random() < 0.25:
        for it in ['beer','diapers']:
            if it not in basket: basket.append(it)
    if rng.random() < 0.3:
        for it in ['coffee','tea']:
            if it not in basket: basket.append(it)
    transactions.append(sorted(set(basket)))
print('Sample transactions (first 5):')
for t in transactions[:5]:
    print(t)

In [ ]:
# 6. Association Rule Mining via Apriori (mlxtend) or fallback implementation
try:
    from mlxtend.frequent_patterns import apriori, association_rules
    from mlxtend.preprocessing import TransactionEncoder
    use_mlxtend = True
except ImportError:
    use_mlxtend = False
    print('mlxtend not installed; will implement a simple apriori fallback.')

if use_mlxtend:
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df_tx = pd.DataFrame(te_ary, columns=te.columns_)
    freq = apriori(df_tx, min_support=0.2, use_colnames=True)
    rules = association_rules(freq, metric='confidence', min_threshold=0.5)
else:
    # Minimal fallback Apriori (not optimized)
    from collections import defaultdict
    def support(itemset):
        count = sum(1 for t in transactions if set(itemset).issubset(t))
        return count / len(transactions)
    single_items = sorted({i for t in transactions for i in t})
    L1 = [{i} for i in single_items if support({i}) >= 0.2]
    freq_itemsets = [({'items': s, 'support': support(s)}) for s in L1]
    # Generate pairs only for brevity
    for i in range(len(L1)):
        for j in range(i+1, len(L1)):
            candidate = L1[i] | L1[j]
            sup = support(candidate)
            if sup >= 0.2:
                freq_itemsets.append({'items': candidate, 'support': sup})
    freq_df = pd.DataFrame([{'itemset': tuple(sorted(fs['items'])), 'support': fs['support']} for fs in freq_itemsets])
    # Simple rule extraction (A->B where itemset size 2)
    rows = []
    for fs in freq_itemsets:
        if len(fs['items']) == 2:
            a, b = tuple(fs['items'])
            sup_ab = fs['support']
            sup_a = support({a})
            sup_b = support({b})
            conf_a_b = sup_ab / sup_a if sup_a else 0
            conf_b_a = sup_ab / sup_b if sup_b else 0
            lift_a_b = conf_a_b / sup_b if sup_b else 0
            lift_b_a = conf_b_a / sup_a if sup_a else 0
            if conf_a_b >= 0.5:
                rows.append({'antecedents': {a}, 'consequents': {b}, 'support': sup_ab, 'confidence': conf_a_b, 'lift': lift_a_b})
            if conf_b_a >= 0.5:
                rows.append({'antecedents': {b}, 'consequents': {a}, 'support': sup_ab, 'confidence': conf_b_a, 'lift': lift_b_a})
    rules = pd.DataFrame(rows)
,

### Task 3 Complete
Delivered classification metrics & visualization, comparative evaluation, synthetic transaction generation, Apriori mining, top rule export, and rule interpretation.